In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork

from moge.visualization.plot_data import matrix_heatmap

Using TensorFlow backend.


In [4]:
tf.enable_eager_execution()

In [2]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)

In [3]:
from moge.network.triplet_generator import OnlineTripletGenerator

generator_train = OnlineTripletGenerator(network, batch_size=256)

Using SampledDataGenerator
Ed_count: 596408 , Eu_count: 151513 , En_count: 755561


/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/edge_generator.py:63: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  self.process_negative_sampling_edges()


Ens_count: 1789224
word index: {'A': 1, 'T': 2, 'G': 3, 'C': 4}
# of nodes to sample from (non-zero degree): 28597


In [53]:
generator_train.batch_size=50

In [5]:
from moge.embedding.siamese_triplet_online_embedding import SiameseTripletGraphEmbedding

siamese = SiameseTripletGraphEmbedding(verbose=True)
siamese.build_keras_model(multi_gpu=False)

Embedding Tensor("embedding_1/embedding_lookup/Identity:0", shape=(?, ?, 4), dtype=float32, device=/device:GPU:0)
conv2D Tensor("lstm_lambda_2/Squeeze:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
max pooling_1 Tensor("max_pooling1d_1/Squeeze:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
conv1d_2 Tensor("lstm_conv_2/Relu:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
max pooling_2 Tensor("max_pooling1d_2/Squeeze:0", shape=(?, ?, 192), dtype=float32, device=/device:GPU:0)
brnn Tensor("bidirectional_1/concat_2:0", shape=(?, 640), dtype=float32, device=/device:GPU:0)
embedding Tensor("embedding_output/BiasAdd:0", shape=(?, 128), dtype=float32, device=/device:GPU:0)
input_seqs Tensor("lstm_network/embedding_output/BiasAdd:0", shape=(2048, 128), dtype=float32, device=/device:GPU:0)
directed_loss Tensor("online_triplet_loss/Mean_2:0", shape=(), dtype=float32, device=/device:GPU:0)
undirected_loss Tensor("online_triplet_loss/Mean_5:0", shape=(), dtype=

AttributeError: 'Model' object has no attribute 'outbound_nodes'

In [62]:
X, y = generator_train.__getitem__(1)

In [75]:
X["labels_directed"]

(matrix([[ 8, 16],
         [14, 39],
         [18, 47],
         [33,  8],
         [33, 16],
         [33, 41],
         [33, 48]], dtype=int32),
 array([1., 1., 1., 1., 1., 1., 1.]),
 (50, 50))

In [76]:
a = tf.SparseTensor(*X["labels_directed"])

In [77]:
tf.sparse_tensor_to_dense( a, default_value = False )

<tf.Tensor: id=131, shape=(50, 50), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [81]:
e = sparse_greater_than(a, c=0.5)
e

In [82]:
e.indices

<tf.Tensor: id=141, shape=(7, 2), dtype=int64, numpy=
array([[ 8, 16],
       [14, 39],
       [18, 47],
       [33,  8],
       [33, 16],
       [33, 41],
       [33, 48]])>

In [83]:
tf.sparse_tensor_to_dense( e, default_value = False )

<tf.Tensor: id=149, shape=(50, 50), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [80]:
def sparse_greater_than(s, c=0.5):
    a = tf.sparse_retain(s, tf.greater(s.values, c))
    return tf.SparseTensor(a.indices, 
                           tf.ones_like(a.values, dtype=tf.bool), 
                           s.dense_shape)

def sparse_not_equal(a, b):
    neg_b = tf.SparseTensor( b.indices, -b.values, dense_shape = b.dense_shape )
    difference = tf.sparse_add( a, neg_b )
    return sparse_not_equal_to_constant( difference, 0.0 )

# # test data
# a = tf.SparseTensor( [ [ 0, 0 ], [ 1, 4 ], [ 2, 3 ] ], [ 5.0, 6, 7 ], 
#                     dense_shape = ( 5, 5 ) )
# b = tf.SparseTensor( [ [ 0, 0 ], [ 0, 2 ], [ 2, 3 ] ], [ 5.0, 6, 2 ], 
#                     dense_shape = ( 5, 5 ) )
# e = sparse_not_equal( a, b )
# f = tf.sparse_tensor_to_dense( e, default_value = False )
# f

In [ ]:
tf.random.learned_unigram_candidate_sampler()

In [ ]:
tf.nn.learned_unigram_candidate_sampler(true_classes=,
    num_true=,
    num_sampled=1,
    unique=False,
    range_max=)